In [ ]:
from tqdm import tqdm
import numpy as np
import os
import pandas as pd

filenames = os.listdir('/content/drive/MyDrive/Colab/Dacon/North_Pole/train/')
filenames.sort()

In [ ]:
data = []

for filename in tqdm(filenames):
    data.append(np.load('/content/drive/MyDrive/Colab/Dacon/North_Pole/train/' + filename))

data = np.array(data)

In [ ]:
import matplotlib.pyplot as plt
plt.subplot(121)
plt.imshow(data[0][:,:,0])

# 북극점을 bbox로 잘라낸 후
# 관측 사진의 빈 부분을 채워줘야 함 -> bbox의 값 중 0인 자리의 값들의 평균으로 채우는 것을 고려중
plt.subplot(122)
plt.imshow(data[0][:,:,1])

In [ ]:
# 해안선 결측값 측정
coastline = np.zeros(shape=data[0][:,:,2].shape)
for i in tqdm(range(data.shape[0])):
    coastline += data[i][:,:,2]
    
coastline = (coastline / data.shape[0]).astype(int)

# 육지 결측값 측정
land = np.zeros(shape=data[0][:,:,3].shape)
for i in tqdm(range(data.shape[0])):
    land += data[i][:,:,3]
    
land = (land / data.shape[0]).astype(int)

# 결측값 측정
missing_value = np.zeros(shape=data[0][:,:,4].shape)
for i in tqdm(range(data.shape[0])):
    missing_value += data[i][:,:,4]

missing_value = (missing_value / data.shape[0]).astype(int)

print((coastline - data[i][:,:,2]).sum())
print((land - data[i][:,:,3]).sum())
print((missing_value - data[i][:,:,4]).sum())

In [ ]:
# 가로
# data[0][:,:,1][447]
# 세로
# data[0][:,:,1][:,303]

# data[0][:,:,1][i].sum() == 0 -> 관측됐다는 의미
# min(row) 인덱스부터 max(row) 인덱스까지 북극점으로 인해 관측이 안됨
# print(min(row), max(row))
# print(min(col), max(col))

for k in tqdm(range(data.shape[0])):
    # 북극점으로 인해 관측이 안된 부분들을 bbox를 만들어 추출
    # 북극점으로 인해 관측이 안된 부분들이 있다면 리스트에 삽입
    # row, col을 통해 xmin, ymin, xmax, ymax를 구할 수 있음
    row = []
    for i in range(data.shape[1]):
        if data[k][:,:,1][i].sum() != 0:
            row.append(i)

    col = []
    for j in range(data.shape[2]):
        if data[k][:,:,1][:,j].sum() != 0:
            col.append(j)
            
    # 북극점으로 관측 안된 부분들은 1로 표현되므로 그 픽셀 값들의 합을 구해줌
    pole = data[k][:,:,1][min(row):max(row)+1,min(col):max(col)+1].sum()
    # row, col을 통해 xmin, ymin, xmax, ymax를 구할 수 있음
    bbox = data[k][:,:,0][min(row):max(row)+1,min(col):max(col)+1].shape
    # 북극점으로 인해 관측 안된 부분은 bbox 중에서 관측된 부분들의 평균으로 대체
    mean_pixel = data[k][:,:,0][min(row):max(row)+1,min(col):max(col)+1].sum() / (bbox[0] * bbox[1] - pole)

    for i in range(bbox[0]):
        for j in range(bbox[1]):
            if data[k][:,:,0][min(row):max(row)+1,min(col):max(col)+1][i][j] == 0:
                data[k][:,:,0][min(row):max(row)+1,min(col):max(col)+1][i][j] = int(mean_pixel)

In [ ]:
data[:,:,:,0].shape
plt.imshow(data[:,:,:,0][0])